# Movies Reviews Sentiment Analysis
### Data

In [1]:
%load_ext autoreload
%autoreload 2
# Cargamos el dataset
from util import load_datasets_unlabeled_test
train, dev, test = load_datasets_unlabeled_test(test_size_val=0) # en realidad dev_size_val
X_train, y_train = train
X_dev, y_dev = dev
X_test=test  # No tenemos el correspondiente y_test

In [2]:
# Vamos a hacer GridSearch con crossvalidation asique unimos los set de training y dev
X_dev

[]

## GridSearchCV - TfidfVectorizer Approach

In [4]:
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn import metrics
from util import eval
from util import iterate_pipeline_params    
import pandas as pd
import numpy as np

### LogisticRegression

In [35]:
param_grid = {
    'vect__binary': [True],
    'vect__ngram_range': [(1, 1)],
    'vect__min_df': [1],
    'vect__max_df': np.linspace(0.2,0.3,5),
    'vect__use_idf': [True],  
    'vect__sublinear_tf': [True], 
    'vect__smooth_idf': [False], 
    'clf__random_state': [0],
    'clf__C': np.linspace(4,8,5),
}


In [36]:
from sklearn.linear_model import LogisticRegression
from util import print_GridSearch_params
#
vect = TfidfVectorizer()
clf = LogisticRegression()

pipeline = Pipeline([
    ('vect', vect),
    ('clf', clf),
])
#
grid = GridSearchCV(pipeline, cv=5, n_jobs=-1, param_grid=param_grid, scoring='accuracy')
grid.fit(X_train, y_train)
# Print GridSearch
print_GridSearch_params(grid,'TRAIN_DEV_CV',fullprint=False)
print('GRID - BEST SCORE')
print(grid.best_score_)

TRAIN_DEV_CV
Mejor conjunto de parámetros para GridSearch en TRAIN_DEV_CV :
{'clf__C': 7.0, 'clf__random_state': 0, 'vect__binary': True, 'vect__max_df': 0.25, 'vect__min_df': 1, 'vect__ngram_range': (1, 1), 'vect__smooth_idf': False, 'vect__sublinear_tf': True, 'vect__use_idf': True}

GRID - BEST SCORE
0.8710280373831776


In [19]:
from util import  load_model,save_model
save_flag=False
if save_flag:
    save_model(grid,'pipe_tv_logreg_GSCV')

### LinearSVC

In [184]:
param_grid = {
    'vect__binary': [True],
    'vect__ngram_range': [(1, 2)],
    'vect__min_df': [3],
    'vect__max_df': [0.95], #[0.39666]
    'vect__use_idf': [True],  
    'vect__sublinear_tf': [False],
    'clf__random_state': [0],
    'clf__C': [2.3]#[1.4210526315789473]
}


In [185]:
from sklearn.svm import LinearSVC
#
vect = TfidfVectorizer()
clf = LinearSVC()

pipeline = Pipeline([
    ('vect', vect),
    ('clf', clf),
])
#
grid = GridSearchCV(pipeline, cv=10, n_jobs=-1, param_grid=param_grid, scoring='accuracy')
grid.fit(X_train, y_train)
# Print GridSearch
print_GridSearch_params(grid,'TRAIN_DEV_CV',fullprint=False)
print('GRID - BEST SCORE')
print(grid.best_score_)

TRAIN_DEV_CV
Mejor conjunto de parámetros para GridSearch en TRAIN_DEV_CV :
{'clf__C': 2.3, 'clf__random_state': 0, 'vect__binary': True, 'vect__max_df': 0.95, 'vect__min_df': 3, 'vect__ngram_range': (1, 2), 'vect__sublinear_tf': False, 'vect__use_idf': True}

GRID - BEST SCORE
0.891588785046729


In [186]:
from util import  load_model,save_model
save_flag=True
if save_flag:
    save_model(grid,'pipe_tv_linearsvc_GSCV_iter')